In [2]:
import pdfplumber
import re
import json
import os
import PyPDF2


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [3]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [4]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

In [5]:
def leggi_dati_da_neo4j(tx) -> Tuple[Dict[str, str], List[str]]:
    """
    Legge macrotecniche e tecniche da Neo4j.
    
    Args:
        tx: Transazione Neo4j
        
    Returns:
        Tuple[Dict[str, str], List[str]]: 
            - Dizionario delle macrotecniche (nome -> descrizione)
            - Lista dei nomi delle tecniche
    """
    # Query per le macrotecniche
    query_macro = """
    MATCH (m:Macrotecnica)
    RETURN m.nome as nome, m.descrizione as descrizione
    """
    result_macro = tx.run(query_macro)
    macrotecniche = {record["nome"]: record["descrizione"] for record in result_macro}
    
    # Query per le tecniche
    query_tecniche = """
    MATCH (t:Tecnica)
    RETURN t.nome as nome
    """
    result_tecniche = tx.run(query_tecniche)
    tecniche = [record["nome"] for record in result_tecniche]
    
    return macrotecniche, tecniche

def associa_tecniche_a_macrotecniche(
    llm: AzureChatOpenAI,
    macrotecniche: Dict[str, str],
    tecniche: List[str]
) -> List[Tuple[str, str]]:
    """
    Associa ogni tecnica alla macrotecnica più simile semanticamente.
    
    Args:
        llm (AzureChatOpenAI): Istanza del modello linguistico
        macrotecniche (Dict[str, str]): Dizionario nome_macrotecnica -> descrizione
        tecniche (List[str]): Lista dei nomi delle tecniche
        
    Returns:
        List[Tuple[str, str]]: Lista di tuple (tecnica, macrotecnica_associata)
    """
    prompt = """
    Data la seguente lista di macrotecniche con le loro descrizioni:
    
    {macro_desc}
    
    E la seguente lista di tecniche:
    
    {tech_list}
    
    Analizza ogni tecnica e associala alla macrotecnica più appropriata basandoti sulla somiglianza semantica e sul contesto.
    Ogni tecnica deve essere associata a una e una sola macrotecnica.
    
    Restituisci un JSON array di oggetti, dove ogni oggetto ha:
    - "tecnica": nome della tecnica
    - "macrotecnica": nome della macrotecnica associata
    - "motivazione": breve spiegazione del perché questa associazione è la migliore
    
    Restituisci SOLO il JSON, senza formattazione markdown o decoratori.
    """
    
    # Formatta le macrotecniche per il prompt
    macro_desc = "\n".join([
        f"- {nome}: {descrizione}"
        for nome, descrizione in macrotecniche.items()
    ])
    
    # Formatta le tecniche per il prompt
    tech_list = "\n".join([f"- {t}" for t in tecniche])
    
    # Invoca il modello
    response = llm.invoke(
        prompt.format(macro_desc=macro_desc, tech_list=tech_list)
    )
    content = response.content.strip()
    
    # Pulizia del JSON
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('json'):
        content = content[4:].strip()
    
    try:
        associazioni = json.loads(content)
    except json.JSONDecodeError:
        print("Errore nel parsing JSON delle associazioni:", content)
        return []
    
    # Estrai solo le coppie tecnica-macrotecnica
    return [(a['tecnica'], a['macrotecnica']) for a in associazioni]

def aggiorna_relazioni_neo4j(tx, associazioni: List[Tuple[str, str]]):
    """
    Aggiorna le relazioni tra tecniche e macrotecniche in Neo4j.
    
    Args:
        tx: Transazione Neo4j
        associazioni: Lista di tuple (tecnica, macrotecnica)
    """
    query = """
    // Prima rimuove tutte le relazioni esistenti per la tecnica
    MATCH (t:Tecnica {nome: $tecnica})-[r:APPARTIENE_A]->(m:Macrotecnica)
    DELETE r
    
    // Poi crea la nuova relazione
    WITH t
    MATCH (m:Macrotecnica {nome: $macrotecnica})
    MERGE (t)-[:APPARTIENE_A]->(m)
    """
    
    for tecnica, macrotecnica in associazioni:
        tx.run(query, tecnica=tecnica, macrotecnica=macrotecnica)

def esegui_matching_completo(driver, llm: AzureChatOpenAI) -> List[Tuple[str, str]]:
    """
    Esegue il processo completo di matching leggendo i dati da Neo4j.
    
    Args:
        driver: Driver Neo4j
        llm (AzureChatOpenAI): Istanza del modello linguistico
        
    Returns:
        List[Tuple[str, str]]: Lista delle associazioni create
    """
    with driver.session() as session:
        # Legge i dati
        macrotecniche, tecniche = session.execute_read(leggi_dati_da_neo4j)
        
        # Fa il matching
        associazioni = associa_tecniche_a_macrotecniche(llm, macrotecniche, tecniche)
        
        # Salva le relazioni
        session.execute_write(aggiorna_relazioni_neo4j, associazioni)
        
        return associazioni

In [7]:
def aggiorna_relazioni_neo4j(tx, associazioni: List[Tuple[str, str]]):
    """
    Aggiorna le relazioni tra tecniche e macrotecniche in Neo4j con verifica.
    
    Args:
        tx: Transazione Neo4j
        associazioni: Lista di tuple (tecnica, macrotecnica)
    """
    for tecnica, macrotecnica in associazioni:
        # Prima verifichiamo che entrambi i nodi esistano
        verifica_query = """
        MATCH (t:Tecnica {nome: $tecnica})
        MATCH (m:Macrotecnica {nome: $macrotecnica})
        RETURN count(t) as tecnica_exists, count(m) as macro_exists
        """
        
        result = tx.run(verifica_query, tecnica=tecnica, macrotecnica=macrotecnica)
        counts = result.single()
        
        if not counts or counts['tecnica_exists'] == 0 or counts['macro_exists'] == 0:
            print(f"ERRORE: Non trovo i nodi per {tecnica} -> {macrotecnica}")
            continue
            
        # Se esistono, procediamo con l'aggiornamento
        update_query = """
        MATCH (t:Tecnica {nome: $tecnica})
        MATCH (m:Macrotecnica {nome: $macrotecnica})
        // Rimuove relazioni esistenti
        OPTIONAL MATCH (t)-[r:APPARTIENE_A]->(:Macrotecnica)
        DELETE r
        // Crea la nuova relazione
        CREATE (t)-[:APPARTIENE_A]->(m)
        RETURN t.nome as tecnica, m.nome as macrotecnica
        """
        
        result = tx.run(update_query, tecnica=tecnica, macrotecnica=macrotecnica)
        record = result.single()
        if record:
            print(f"SUCCESS: Creata relazione {record['tecnica']} -> {record['macrotecnica']}")
        else:
            print(f"ERRORE: Fallita creazione relazione per {tecnica} -> {macrotecnica}")

def esegui_matching_completo(driver, llm: AzureChatOpenAI) -> List[Tuple[str, str]]:
    """
    Esegue il processo completo di matching con logging dettagliato.
    
    Args:
        driver: Driver Neo4j
        llm (AzureChatOpenAI): Istanza del modello linguistico
    """
    with driver.session() as session:
        # Legge i dati
        print("Lettura dati da Neo4j...")
        macrotecniche, tecniche = session.execute_read(leggi_dati_da_neo4j)
        
        print(f"Trovate {len(macrotecniche)} macrotecniche e {len(tecniche)} tecniche")
        print("\nMacrotecniche trovate:")
        for nome, desc in macrotecniche.items():
            print(f"- {nome}")
        
        print("\nTecniche trovate:")
        for t in tecniche:
            print(f"- {t}")
            
        # Fa il matching
        print("\nEsecuzione matching...")
        associazioni = associa_tecniche_a_macrotecniche(llm, macrotecniche, tecniche)
        
        print(f"\nCreate {len(associazioni)} associazioni:")
        for tecnica, macro in associazioni:
            print(f"- {tecnica} -> {macro}")
        
        # Salva le relazioni
        print("\nSalvataggio relazioni in Neo4j...")
        session.execute_write(aggiorna_relazioni_neo4j, associazioni)
        
        # Verifica finale
        print("\nVerifica delle relazioni create:")
        verifica_query = """
        MATCH (t:Tecnica)-[r:APPARTIENE_A]->(m:Macrotecnica)
        RETURN t.nome as tecnica, m.nome as macrotecnica
        """
        result = session.run(verifica_query)
        relazioni_esistenti = [(record["tecnica"], record["macrotecnica"]) for record in result]
        
        print(f"Trovate {len(relazioni_esistenti)} relazioni nel database:")
        for tecnica, macro in relazioni_esistenti:
            print(f"- {tecnica} -> {macro}")
            
        return associazioni

# Query di test utili:
"""
# Per vedere tutte le tecniche:
MATCH (t:Tecnica) RETURN t.nome

# Per vedere tutte le macrotecniche:
MATCH (m:Macrotecnica) RETURN m.nome, m.descrizione

# Per vedere tutte le relazioni esistenti:
MATCH (t:Tecnica)-[r:APPARTIENE_A]->(m:Macrotecnica)
RETURN t.nome as tecnica, m.nome as macrotecnica

# Per contare le relazioni:
MATCH (t:Tecnica)-[r:APPARTIENE_A]->(m:Macrotecnica)
RETURN count(r) as num_relazioni
"""

'\n# Per vedere tutte le tecniche:\nMATCH (t:Tecnica) RETURN t.nome\n\n# Per vedere tutte le macrotecniche:\nMATCH (m:Macrotecnica) RETURN m.nome, m.descrizione\n\n# Per vedere tutte le relazioni esistenti:\nMATCH (t:Tecnica)-[r:APPARTIENE_A]->(m:Macrotecnica)\nRETURN t.nome as tecnica, m.nome as macrotecnica\n\n# Per contare le relazioni:\nMATCH (t:Tecnica)-[r:APPARTIENE_A]->(m:Macrotecnica)\nRETURN count(r) as num_relazioni\n'

In [8]:
# Esegue l'intero processo
associazioni = esegui_matching_completo(driver, llm)

for tecnica, macro in associazioni:
    print(f"{tecnica} -> {macro}")

Lettura dati da Neo4j...
Trovate 29 macrotecniche e 70 tecniche

Macrotecniche trovate:
- Fermentazione
- Tecniche di Impasto
- Surgelamento
- Bollitura
- Grigliare
- Forno
- Vapore
- Sottovuoto
- Saltare in Padella
- Decostruzione
- Decostruzione Atomica a Strati Energetici
- Decostruzione Magnetica Risonante
- Decostruzione Bio-Fotonica Emotiva
- Decostruzione Ancestrale
- Decostruzione Interdimensionale Lovecraftiana
- Sferificazione
- Sferificazione a Gravità Psionica Variabile
- Sferificazione Filamentare a Molecole Vibrazionali
- Sferificazione Cromatica Interdimensionale
- Sferificazione con Campi Magnetici Entropici
- Sferificazione tramite Matrici Biofotiche
- Tecniche di Taglio
- Taglio Dimensionale a Lame Fotofiliche
- Taglio a Risonanza Sonica Rigenerativa
- Affettamento a Pulsazioni Quantistiche
- Taglio Sinaptico Biomimetico
- Incisione Elettromagnetica Plasmica
- Marinatura
- Affumicatura

Tecniche trovate:
- Marinatura a Infusione Gravitazionale
- Marinatura Temporale S